In [10]:
import torch
import torch.nn as nn
import  torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import os

In [11]:
train_dir=r"E:\\HTM\\zephyr\\Images11\\train"
test_dir=r"E:\\HTM\\zephyr\\Images11\\test"

In [12]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [13]:
train_data=datasets.ImageFolder(train_dir,transform=transform)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=64,shuffle=True)

test_data=datasets.ImageFolder(test_dir,transform=transform)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=64,shuffle=True)

In [14]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
device

device(type='cuda')

In [16]:
class Colornet(nn.Module):
    def __init__(self):
        super(Colornet,self).__init__()
        self.conv1=nn.Conv2d(1,64,kernel_size=5,stride=1,padding=4,dilation=2)
        self.conv2=nn.Conv2d(64,64,kernel_size=5,stride=1,padding=4,dilation=2)
        self.conv3=nn.Conv2d(64,128,kernel_size=5,stride=1,padding=4,dilation=2)
        self.conv4=nn.Conv2d(128,3,kernel_size=5,stride=1,padding=4,dilation=2)

    def forward(self,x):
        x=nn.functional.relu(self.conv1(x))
        x=nn.functional.relu(self.conv2(x))
        x=nn.functional.relu(self.conv3(x))
        x=torch.sigmoid(self.conv4(x))
        return x

In [17]:
model = Colornet().to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert RGB image to grayscale
def rgb_to_gray(img):
    return img.mean(dim=1, keepdim=True)


In [18]:
# Training loop
EPOCHS = 1
for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(train_loader):
        grayscale_images = rgb_to_gray(images).to(device)
        images = images.to(device)

        # Forward pass
        outputs = model(grayscale_images)
        loss = criterion(outputs, images)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print statistics
        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

print("Finished Training")


Epoch [1/1], Step [1/225], Loss: 0.0655
Epoch [1/1], Step [101/225], Loss: 0.0062
Epoch [1/1], Step [201/225], Loss: 0.0037


In [3]:
from PIL import Image
img = Image.open("E:\\HTM\\bwAgri.jpg")
gray_img = img.convert("L")
gray_img.save("./bwsar23.jpeg")
img_tensor = transform(gray_img).unsqueeze(0)
model.eval()
img_tensor = img_tensor.to(device)
with torch.no_grad():
    colorized_tensor = model(img_tensor)

NameError: name 'transform' is not defined

In [32]:
colorized_img = transforms.ToPILImage()(colorized_tensor.squeeze(0).cpu())
colorized_img.save("./colorizedsar23LL.jpeg")

In [13]:
saving_path="E:\\HTM\\model.cpkt"
torch.save(model,saving_path)